In [1]:
import seaborn as sns
import os
import glob
import re
import pandas as pd
from matplotlib import pyplot as plt
os.getcwd()

'/home/owaismujtaba/projects/mind-voice'

In [2]:
def load_decoding_accuracies_per_class(results_dir):
    """
    Scan `results_dir` for files like:
      sub-13_ses-01_classification_report.csv

    Extract precision, recall, f1 for each class from each file.

    Parameters
    ----------
    results_dir : str
        Path to the folder containing *_classification_report.csv files.

    Returns
    -------
    pd.DataFrame
        Columns: ['subject_id', 'session_id', 'class_id', 'precision', 'recall', 'f1']
    """
    pattern = os.path.join(results_dir, "*_classification_report.csv")
    files = glob.glob(pattern)

    filename_re = re.compile(r"sub-(\d+)_ses-(\d+)_classification_report\.csv")
    records = []

    for filepath in files:
        fname = os.path.basename(filepath)
        m = filename_re.match(fname)
        if not m:
            continue

        subject_id, session_id = m.groups()

        # Load CSV assuming no header in first column (which holds class label)
        df = pd.read_csv(filepath)
        
        # Rename the first column (index column) to 'class'
        df.rename(columns={df.columns[0]: 'class'}, inplace=True)

        # Keep only rows where 'class' is a digit (i.e., individual classes)
        df = df[df['class'].apply(lambda x: str(x).isdigit())].copy()

        for _, row in df.iterrows():
            class_id = int(row['class'])
            records.append({
                "subject_id": subject_id,
                "session_id": session_id,
                "class_id": class_id,
                "precision": float(row['precision']),
                "recall": float(row['recall']),
                "f1": float(row['f1-score'])
            })

    return pd.DataFrame(records)


# Example usage:
df = load_decoding_accuracies_per_class("results/DecodingResults")

In [3]:
df

,subject_id,session_id,class_id,precision,recall,f1
0,05,02,0,0.819672,0.980392,0.892857
1,05,02,1,0.747748,0.813725,0.779343
2,05,02,2,0.835616,0.598039,0.697143
3,11,01,0,0.728682,0.921569,0.813853
4,11,01,1,0.718750,0.676471,0.696970
...,...,...,...,...,...,...
61,03,01,1,0.680851,0.941176,0.790123
62,03,01,2,0.855072,0.578431,0.690058
63,13,02,0,0.773438,0.970588,0.860870
64,13,02,1,0.772727,0.833333,0.801887


In [4]:
precision = df.groupby(by=['subject_id', 'session_id','class_id'])['precision'].mean().reset_index()
precision = precision.groupby(by='class_id')['precision'].mean()
precision

class_id
0    0.782048
1    0.772887
2    0.783120
Name: precision, dtype: float64

In [5]:
recall = df.groupby(by=['subject_id', 'session_id','class_id'])['recall'].mean().reset_index()
recall = recall.groupby(by='class_id')['recall'].mean()
recall

class_id
0    0.913136
1    0.711709
2    0.633101
Name: recall, dtype: float64

In [6]:
f1 = df.groupby(by=['subject_id', 'session_id','class_id'])['f1'].mean().reset_index()
f1 = f1.groupby(by='class_id')['f1'].mean()
f1

class_id
0    0.828259
1    0.721097
2    0.686058
Name: f1, dtype: float64

In [7]:
precision = df.groupby(by=['subject_id', 'session_id','class_id'])['precision'].mean().reset_index()
precision_std = precision.groupby(by='class_id')['precision'].std()
precision_std

class_id
0    0.152522
1    0.104383
2    0.102148
Name: precision, dtype: float64

In [8]:
recall = df.groupby(by=['subject_id', 'session_id','class_id'])['recall'].mean().reset_index()
recall_std = recall.groupby(by='class_id')['recall'].std()
recall_std

class_id
0    0.088555
1    0.192325
2    0.162317
Name: recall, dtype: float64

In [9]:
f1 = df.groupby(by=['subject_id', 'session_id','class_id'])['f1'].mean().reset_index()
f1_std = f1.groupby(by='class_id')['f1'].std()
f1_std

class_id
0    0.087085
1    0.115602
2    0.113067
Name: f1, dtype: float64